In [1]:
# 安装 Java（Spark 依赖）
!apt-get update -y
!apt-get install -y openjdk-8-jdk-headless

!pip install pyspark==3.5.5 spark-nlp transformers datasets torch fastapi uvicorn nest-asyncio pyngrok
!pip install -U pandas pyarrow

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Get:13 https://r2u.stat.illinois

In [2]:
!pip install findspark

In [3]:

!apt-get update -qq
!apt-get install -qq openjdk-8-jdk-headless

!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz

!pip install -q findspark


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [4]:
import os
# 指定 Java 和 Spark 根目录
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

import findspark
findspark.init()  

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SentimentAnalysis") \
    .getOrCreate()
print("✅ Spark version:", spark.version)


✅ Spark version: 3.3.2


In [5]:
# English comment: Download the Sentiment140 dataset (1.6M tweets)
!pip install -q kaggle
import kagglehub

path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment140


In [6]:
!unzip archive.zip

Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [7]:
# English comment: Load CSV into Spark DataFrame and select relevant columns
from pyspark.sql.functions import col, when, regexp_replace, lower

df = spark.read.csv("training.1600000.processed.noemoticon.csv", header=False, inferSchema=True) \
    .selectExpr("_c0 as target", "_c5 as text")


df = df.withColumn("label",
    when(col("target") == 0, 0)
   .when(col("target") == 2, 1)
   .otherwise(2)
)


df = df.withColumn("clean_text",
     lower(regexp_replace(col("text"), r"http\S+|@\S+|[^a-zA-Z\s]", ""))
)

df = df.select("clean_text", "label").cache()
train_df, val_df, test_df = df.randomSplit([0.8, 0.1, 0.1], seed=25)

print("Training records:", train_df.count())


Training records: 1279913


In [8]:
!pip install -U pandas pyarrow


In [9]:
# English comment: Convert Spark DataFrames to pandas and save as CSV
train_df.toPandas().to_csv("train.csv", index=False)
val_df.toPandas().to_csv("val.csv", index=False)
test_df.toPandas().to_csv("test.csv", index=False)


In [10]:
# English comment: Load datasets and tokenizer/model
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# English comment: Load CSV files as a Hugging Face dataset
dataset = load_dataset("csv", data_files={
    "train": "train.csv",
    "validation": "val.csv",
    "test": "test.csv"
})

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# English comment: Tokenization function
def preprocess_function(examples):
    return tokenizer(
        examples["clean_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# English comment: Define training arguments
training_args = TrainingArguments(
    output_dir="bert_sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="logs",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
)

# English comment: Compute metrics for evaluation
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        p.label_ids, preds, average="weighted"
    )
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics
)

# English comment: Start training
trainer.train()

# English comment: Evaluate on test set
metrics = trainer.evaluate(tokenized_datasets["test"])
print(metrics)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yuhengliu to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.347000,0.342772,0.852581,0.852748,0.852581,0.852563
2,0.293500,0.333525,0.860228,0.860354,0.860228,0.860215


{'eval_loss': 0.3372812867164612, 'eval_accuracy': 0.8582855286016022, 'eval_precision': 0.8583405219584166, 'eval_recall': 0.8582855286016022, 'eval_f1': 0.8582789284279433, 'eval_runtime': 244.8639, 'eval_samples_per_second': 653.53, 'eval_steps_per_second': 20.424, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")


('sentiment_model/tokenizer_config.json',
 'sentiment_model/special_tokens_map.json',
 'sentiment_model/vocab.txt',
 'sentiment_model/added_tokens.json',
 'sentiment_model/tokenizer.json')

In [17]:
!zip -r sentiment_model.zip sentiment_model

# Step 2: Download the zip file to local machine
from google.colab import files
files.download("sentiment_model.zip")

updating: sentiment_model/ (stored 0%)
updating: sentiment_model/tokenizer_config.json (deflated 75%)
updating: sentiment_model/model.safetensors (deflated 7%)
updating: sentiment_model/special_tokens_map.json (deflated 42%)
updating: sentiment_model/config.json (deflated 51%)
updating: sentiment_model/vocab.txt (deflated 53%)
updating: sentiment_model/tokenizer.json (deflated 71%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
# run_sentiment_test.py

# English comment: This script loads a fine-tuned BERT model and performs sentiment prediction on sample input.

from transformers import BertTokenizerFast, BertForSequenceClassification
import torch

# Load model and tokenizer from local directory
MODEL_DIR = "sentiment_model"  # Update if you saved the model to another path

print("🔧 Loading model and tokenizer...")
tokenizer = BertTokenizerFast.from_pretrained(MODEL_DIR)
model = BertForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()  # Set model to evaluation mode

# Define input text samples
sample_texts = [
    "I absolutely love this product! Highly recommended.",
    "It's okay, not great but not bad either.",
    "This is the worst service I've ever received."
]


label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Run predictions
print("📊 Running sentiment predictions:")
for i, text in enumerate(sample_texts):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_label_idx = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_label_idx].item()

    # Print result
    print(f"\n🔹 Sample {i+1}: \"{text}\"")
    print(f"   → Sentiment: {label_map[pred_label_idx]}")
    print(f"   → Confidence: {confidence:.4f}")


🔧 Loading model and tokenizer...
📊 Running sentiment predictions:

🔹 Sample 1: "I absolutely love this product! Highly recommended."
   → Sentiment: positive
   → Confidence: 0.9950

🔹 Sample 2: "It's okay, not great but not bad either."
   → Sentiment: positive
   → Confidence: 0.7406

🔹 Sample 3: "This is the worst service I've ever received."
   → Sentiment: negative
   → Confidence: 0.9934
